# Neural Network Implementation-From Scratch 

## A. Neural Network: Binary Classification  

In this assignment, we will learn to build a fully-connected neural network with standard architecture, i.e., only one hidden layer. We will use the `Breast cancer wisconsin (diagnostic) dataset` available in `https://scikit-learn.org/stable/datasets/index.html`. It has a total of 569 sample with two classes (Malignant and Benign). Each sample has 30 real-valued features. 

**This assignment will help you understand and implement:**
- A neural network for binary classification consisting of one hidden layer with non-linear activation. You will implement ideas like forward propogation, computing the loss, bagward propogation, and parameter(weights) update. Using the trained network, you will learn to predict a class given the features of a sample.

**Note:** There are multiple conventions for coding neural networks. We will follow the conventions suggested by Andrew Ng: https://www.coursera.org/learn/neural-networks-deep-learning



In [ ]:
# Package imports
import numpy as np
import sklearn
import sklearn.linear_model
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
import matplotlib.pyplot as plt

%matplotlib inline 

np.random.seed(1)

## 1. Loading the dataset and preprocessing

In [ ]:
X, y = load_breast_cancer(return_X_y=True)
train_data, test_data, train_labels, test_labels = train_test_split(X, y, test_size =0.2, random_state=0)

print(X.shape)

(569, 30)


In [ ]:
scaler = preprocessing.StandardScaler().fit(train_data)
train_data = scaler.transform(train_data)
test_data = scaler.transform(test_data)

In [ ]:
trainx = train_data.T
trainy = train_labels.reshape(-1,1).T

testx = test_data.T
testy =test_labels.reshape(-1,1).T

In [ ]:
trainx.shape, trainy.shape, testx.shape, testy.shape

((30, 455), (1, 455), (30, 114), (1, 114))

In [ ]:
X=trainx
Y=trainy

In [ ]:
### START CODE HERE ###
shape_X = trainx.shape
shape_Y = trainy.shape
m = shape_X[1]  # training set size
### END CODE HERE ###

print ('No. of training samples: ' + str(m))
print ('Number of features per sample: ' + str(shape_X[0]))

No. of training samples: 455
Number of features per sample: 30


## 2 - Neural Network model

We will train a Neural Network with a single hidden layer.

**Mathematically**:

For one example $x^{(i)}$:
$$z^{[1] (i)} =  W^{[1]} x^{(i)} + b^{[1]}\tag{1}$$ 
$$a^{[1] (i)} = \tanh(z^{[1] (i)})\tag{2}$$
$$z^{[2] (i)} = W^{[2]} a^{[1] (i)} + b^{[2]}\tag{3}$$
$$\hat{y}^{(i)} = a^{[2] (i)} = \sigma(z^{ [2] (i)})\tag{4}$$
$$y^{(i)}_{prediction} = \begin{cases} 1 & \mbox{if } a^{[2](i)} > 0.5 \\ 0 & \mbox{otherwise } \end{cases}\tag{5}$$

Given the predictions on all the examples, you can also compute the cost (loss) $J$ as follows: 
$$J = - \frac{1}{m} \sum\limits_{i = 0}^{m} \large\left(\small y^{(i)}\log\left(a^{[2] (i)}\right) + (1-y^{(i)})\log\left(1- a^{[2] (i)}\right)  \large  \right) \small \tag{6}$$

**Important**: Building the NN will involve the following:

    1. Specify the network structure in terms of the number of input units, number of neurons in the hidden units, ...
    2. Initialize the parameters of the model
    3. Loop a number of iterations:
        - Forward propagation
        - Compute loss and the overall loss
        - Backward propagation
        - Update the parameters (gradient descent)

In order to make the code modular, we can implement each of the step as a function and them combine them together to build the overall model.

### 2.1 - Specify the network structure 
    - n_x: input layer size
    - n_h: #neurons in  hidden layer (hard code a value, say 10) 
    - n_y: the size of the output layer

In [ ]:
def model_architecture(X, Y):
    """
    Arguments:
    X -- input dataset of shape (input size, number of examples)
    Y -- labels of shape (output size, number of examples)
    
    Returns:
    n_x -- the size of the input layer
    n_h -- the size of the hidden layer
    n_y -- the size of the output layer
    """
    ### START CODE HERE ### 
    n_x = X.shape[0] # size of input layer
    n_h = 3
    n_y = Y.shape[0] # size of output layer
    ### END CODE HERE ###
    return (n_x, n_h, n_y)

### 2.2 - Initialize the parameters of the model

- Initialize the weights matrices with random values. 
    - Use: `np.random.randn(a,b) * 0.01` to randomly initialize a matrix of shape (a,b).
- Initialize the bias vectors as zeros. 
    - Use: `np.zeros((a,b))` to initialize a matrix of shape (a,b) with zeros.

In [ ]:
def initialize_parameters(n_x, n_h, n_y):
    """
    Argument:
    n_x -- size of the input layer
    n_h -- size of the hidden layer
    n_y -- size of the output layer
    
    Returns:
    params -- python dictionary containing your parameters:
                    W1 -- weight matrix of shape (n_h, n_x)
                    b1 -- bias vector of shape (n_h, 1)
                    W2 -- weight matrix of shape (n_y, n_h)
                    b2 -- bias vector of shape (n_y, 1)
    """
    
    np.random.seed(2)

    
    ### START CODE HERE ###
    W1 = np.random.randn(n_h, n_x)*0.01
    b1 = np.zeros((n_h, 1))
    W2 = np.random.randn(n_y, n_h)*0.01
    b2 = np.zeros((n_y, 1))
    ### END CODE HERE ###
    
    assert (W1.shape == (n_h, n_x))
    assert (b1.shape == (n_h, 1))
    assert (W2.shape == (n_y, n_h))
    assert (b2.shape == (n_y, 1))
    
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    
    return parameters

### 2.3 - The Loop ####

**Instructions**:
- Look above at the mathematical representation of your classifier.
- Define the function `sigmoid()`.
- You can use the function `np.tanh()`. It is part of the numpy library.
- The steps you have to implement are:
    1. Retrieve each parameter from the dictionary "parameters" (which is the output of `initialize_parameters()`) by using `parameters[".."]`.
    2. Implement Forward Propagation. Compute $Z^{[1]}, A^{[1]}, Z^{[2]}$ and $A^{[2]}$ (the vector of all your predictions on all the examples in the training set).
- Values needed in the backpropagation are stored in "`cache`". The `cache` will be given as an input to the backpropagation function.

In [ ]:
def sigmoid(x):
    ### Update THE CODE HERE ###
    return 1/(1+np.exp(-x))
    ### END CODE HERE ###

In [89]:
def forward_propagation(X, parameters):
    """
    Argument:
    X -- input data of size (n_x, m)
    parameters -- python dictionary containing your parameters (output of initialization function)
    
    Returns:
    A2 -- The sigmoid output of the second activation
    cache -- a dictionary containing "Z1", "A1", "Z2" and "A2"
    """
    # Retrieve each parameter from the dictionary "parameters"
    ### START CODE HERE ### 
    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']
    ### END CODE HERE ###
    
    # Implement Forward Propagation to calculate A2 (probabilities)
    ### START CODE HERE ### 
    Z1 = np.dot(W1, X) + b1
    A1 = np.tanh(Z1)
    Z2 = np.dot(W2, A1) + b2
    A2 = sigmoid(Z2)
    ### END CODE HERE ###
    
    assert(A2.shape == (1, X.shape[1]))
    
    cache = {"Z1": Z1,
             "A1": A1,
             "Z2": Z2,
             "A2": A2}
    
    return A2, cache

Now you have computed $A^{[2]}$ (in the Python variable "`A2`"), which contains $a^{[2](i)}$ for every example, you can compute the loss function as follows:

$$J = - \frac{1}{m} \sum\limits_{i = 1}^{m} \large{(} \small y^{(i)}\log\left(a^{[2] (i)}\right) + (1-y^{(i)})\log\left(1- a^{[2] (i)}\right) \large{)} \small\tag{7}$$

- Implement the cross-entropy loss:
$- \sum\limits_{i=0}^{m}  y^{(i)}\log(a^{[2](i)})$:
```python
logprobs = np.multiply(np.log(A2),Y)
loss = - np.sum(logprobs)
```

In [ ]:
def compute_loss(A2, Y):
    """
    Arguments:
    A2 -- The sigmoid output of the second activation, of shape (1, number of examples)
    Y -- "true" labels vector of shape (1, number of examples)
       
    Returns:
    loss -- cross-entropy loss given equation (7)
    
    """
    
    m = Y.shape[1] # number of example

    # Compute the cross-entropy loss
    ### START CODE HERE ###
    logprobs = np.multiply(np.log(A2), Y) + np.multiply((1-Y), np.log(1 - A2))
    loss = - np.sum(logprobs) / m
    ### END CODE HERE ###
    
    loss = float(np.squeeze(loss))

    assert(isinstance(loss, float))
    
    return loss

Using the cache computed during forward propagation, we can now implement backward propagation.

Backpropagation is usually the hardest (most mathematical) part. You can use the following six equations as vectorized implementation:

$$dZ^{[2]} = A^{[2]} - Y \tag{8}$$ 
$$dW^{[2]} = \frac{1}{m} dZ^{[2]}A^{[1]{T}} \tag{9}$$ 
$$db^{[2]} = \frac{1}{m} np.sum(dZ^{[2]}, axis = 1, keepdims = True)\tag{10}$$ 
$$dZ^{[1]} = W^{[2]T}dZ^{[2]}*g^{[1]'}(Z^{[1]}) \tag{11}$$ 
$$dW^{[1]} = \frac{1}{m} dZ^{[1]}X^{{T}} \tag{12}$$ 
$$db^{[1]} = \frac{1}{m} np.sum(dZ^{[1]}, axis = 1, keepdims = True)\tag{13}$$ 

- $*$ denotes elementwise multiplication.
- Notations followed:
    - dW1 = $\frac{\partial \mathcal{J} }{ \partial W_1 }$
    - db1 = $\frac{\partial \mathcal{J} }{ \partial b_1 }$
    - dW2 = $\frac{\partial \mathcal{J} }{ \partial W_2 }$
    - db2 = $\frac{\partial \mathcal{J} }{ \partial b_2 }$
   
- Tips:
    - To compute dZ1 you'll need to compute $g^{[1]'}(Z^{[1]})$. Since $g^{[1]}(.)$ is the tanh activation function, if $a = g^{[1]}(z)$ then $g^{[1]'}(z) = 1-a^2$. So you can compute 
    $g^{[1]'}(Z^{[1]})$ using `(1 - np.power(A1, 2))`.

In [ ]:
def backprop(parameters, cache, X, Y):
    """
    Arguments:
    parameters -- python dictionary containing our parameters 
    cache -- a dictionary containing "Z1", "A1", "Z2" and "A2".
    X -- input data
    Y -- "true" labels
    
    Returns:
    grads -- python dictionary containing your gradients with respect to different parameters
    """
    m = X.shape[1]
    
    # First, retrieve W1 and W2 from the dictionary "parameters".
    ### START CODE HERE ### 
    W1 = parameters['W1']
    W2 = parameters['W2']
    ### END CODE HERE ###
        
    # Retrieve also A1 and A2 from dictionary "cache".
    ### START CODE HERE ### 
    A1 = cache['A1']
    A2 = cache['A2']
    ### END CODE HERE ###
    
    # Backward propagation: calculate dW1, db1, dW2, db2. 
    ### START CODE HERE ### 
    dZ2 = A2-Y
    dW2 = (1/m) * np.dot(dZ2, A1.T)
    db2 = (1/m) * np.sum(dZ2, axis=1, keepdims=True)
    dZ1 = np.multiply(np.dot(W2.T, dZ2), 1 - np.power(A1, 2))
    dW1 = (1/m) * np.dot(dZ1, X.T) 
    db1 = (1/m)*np.sum(dZ1, axis=1, keepdims=True)
    ### END CODE HERE ###
    
    grads = {"dW1": dW1,
             "db1": db1,
             "dW2": dW2,
             "db2": db2}
    
    return grads

Implement the update rule. Use gradient descent. You have to use (dW1, db1, dW2, db2) in order to update (W1, b1, W2, b2).

**Gradient descent rule**: $ \theta = \theta - \alpha \frac{\partial J }{ \partial \theta }$ where $\alpha$ is the learning rate and $\theta$ represents a parameter.

**Illustration**: The gradient descent algorithm with a good learning rate (converging) and a bad learning rate (diverging). Images courtesy of Adam Harley.

<img src="images\sgd.gif" style="width:400;height:400;"> <img src="images\sgd_bad.gif" style="width:400;height:400;">



In [ ]:
def update(parameters, grads, learning_rate = 0.01):
    """
    Arguments:
    parameters -- python dictionary containing your parameters 
    grads -- python dictionary containing your gradients 
    learning_rate -- The learning rate
    
    Returns:
    parameters -- python dictionary containing your updated parameters 
    """
    # Retrieve each parameter from the dictionary "parameters"
    ### START CODE HERE ### 
    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']
    ### END CODE HERE ###
    
    # Retrieve each gradient from the dictionary "grads"
    ### START CODE HERE ### 
    dW1 = grads['dW1']
    db1 = grads['db1']
    dW2 = grads['dW2']
    db2 = grads['db2']
    ## END CODE HERE ###
    
    # Update rule for each parameter
    ### START CODE HERE ### 
    W1 = W1 - learning_rate * dW1
    b1 = b1 - learning_rate * db1
    W2 = W2 - learning_rate * dW2
    b2 = b2 - learning_rate * db2
    ### END CODE HERE ###
    
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    
    return parameters

### 2.4 - Integrate parts 2.1, 2.2 and 2.3 in NeuralNetwork() ####

Build your neural network model in `NeuralNetwork()`.

**Instructions**: The neural network model has to use the previous functions in the right order.

In [ ]:
def NeuralNetwork(X, Y, n_h, num_iterations = 10000, learning_rate = 0.01, print_loss=False):
    """
    Arguments:
    X -- dataset
    Y -- labels 
    n_h -- size of the hidden layer
    num_iterations -- Number of iterations in gradient descent loop
    learning_rate -- The learning rate
    print_loss -- if True, print the loss every 1000 iterations
    
    Returns:
    parameters -- parameters learnt by the model. They can then be used to make predictions.
    """
    
    np.random.seed(3)
    n_x = model_architecture(X, Y)[0]
    n_y = model_architecture(X, Y)[2]
    
    # Initialize parameters
    ### START CODE HERE ### 
    parameters = initialize_parameters(n_x, n_h, n_y)
    ### END CODE HERE ###
    
    # Loop (gradient descent)

    for i in range(0, num_iterations):
         
        ### START CODE HERE ### 
        # Forward propagation. Inputs: "X, parameters". Outputs: "A2, cache".
        A2, cache = forward_propagation(X, parameters)
        
        # loss function. Inputs: "A2, Y, parameters". Outputs: "loss".
        loss = compute_loss(A2, Y)
 
        # Backpropagation. Inputs: "parameters, cache, X, Y". Outputs: "grads".
        grads = backprop(parameters, cache, X, Y)
 
        # Gradient descent parameter update. Inputs: "parameters, grads". Outputs: "parameters".
        parameters =  update(parameters, grads)
        
        ### END CODE HERE ###
        
        # Print the loss every 100 iterations
        if print_loss and i % 100 == 0:
            print ("loss after iteration %i: %f" %(i, loss))

    return parameters

### 2.5 Predictions

Use your model to predict by building predict().

**Reminder**: $ predictions = \begin{cases}
      1 & \text{if}\ activation > 0.5 \\
      0 & \text{otherwise}
    \end{cases}$  
    
As an example, if you would like to set the entries of a matrix X to 0 and 1 based on a threshold you would do: ```X_new = (X > threshold)```

In [ ]:
def predict(parameters, X):
    """
    Arguments:
    parameters -- python dictionary containing your parameters 
    X -- input data 
    
    Returns
    predictions -- vector of predictions of our model
    """
    
    # Computes probabilities using forward propagation, and classifies to 0/1 using 0.5 as the threshold.
    ### START CODE HERE ### 
    A2, cache = forward_propagation(X, parameters)
    predictions = A2 > 0.5
    ### END CODE HERE ###
    
    return predictions

## 3. Model Execution
It is time to run the model and see how it performs on the dataset. Run the following code to test your model with a single hidden layer of $n_h$ hidden units.

In [93]:
# Build a model with a n_h-dimensional hidden layer
parameters = NeuralNetwork(X, Y, n_h = 4, num_iterations = 10000, print_loss=True)

loss after iteration 0: 0.693498
loss after iteration 100: 0.677406
loss after iteration 200: 0.657364
loss after iteration 300: 0.589216
loss after iteration 400: 0.468793
loss after iteration 500: 0.357196
loss after iteration 600: 0.275707
loss after iteration 700: 0.220609
loss after iteration 800: 0.183567
loss after iteration 900: 0.157976
loss after iteration 1000: 0.139656
loss after iteration 1100: 0.126086
loss after iteration 1200: 0.115729
loss after iteration 1300: 0.107618
loss after iteration 1400: 0.101124
loss after iteration 1500: 0.095823
loss after iteration 1600: 0.091425
loss after iteration 1700: 0.087722
loss after iteration 1800: 0.084565
loss after iteration 1900: 0.081841
loss after iteration 2000: 0.079469
loss after iteration 2100: 0.077385
loss after iteration 2200: 0.075540
loss after iteration 2300: 0.073894
loss after iteration 2400: 0.072417
loss after iteration 2500: 0.071084
loss after iteration 2600: 0.069875
loss after iteration 2700: 0.068773
loss

## Check the accuracy on the training set

In [94]:
# Print accuracy
predictions = predict(parameters, X)
print(accuracy_score(Y.T, predictions.T))

0.989010989010989


## Check the accuracy on the test set

In [95]:
predictions_test = predict(parameters, testx) 
print(accuracy_score(testy.T, predictions_test.T))

0.9736842105263158


## Run the model multiple times with different hyperparameters and write your interpretation


Write your interpretation here

1-The model has been run with different hyperparameters as follows:
 - Iterations number = 10000, resulted in: traing accuracy =  0.989010989010989 and testing accuracy = 0.9736842105263158

 - Iterations number = 1000, resulted in: traing accuracy =  0.978021978021978 and testing accuracy = 0.956140350877193 

 - Iterations number = 100, resulted in: traing accuracy =  0.6373626373626373 and testing accuracy = 0.5877192982456141


 - Hidden layer = 4, resulted in: traing accuracy =  0.989010989010989 and testing accuracy = 0.9736842105263158

 - Hidden layer = 10, resulted in: traing accuracy =  0.989010989010989 and testing accuracy = 0.9649122807017544

 - Hidden layer = 50, resulted in: traing accuracy =  0.9912087912087912 and testing accuracy = 0.9649122807017544

  


What happens when you change the tanh activation for a sigmoid activation or a ReLU activation?

 - As the first experiment with 1000 iterations showed that with using tanh activation function, the loss reach to: 0.157976, with training accuracy reach to: 0.978021978021978, and the testing accuracy reach to: 0.956140350877193 

 -The experiment using sigmoid activation function with 1000 iterations showed that, the loss was a little bit heigher to reach to: 0.341950, with training accuracy dropped a little bit to reach to: 0.9384615384615385, as well as testing accuracy to reach to:0.9298245614035088

 - The last experiment with 1000 iterations showed that with using relu activation function, the loss reach to: 0.338804, with training accuracy dropped a little bit to reach to: 0.9472527472527472, as well as testing accuracy to each to: 0.9210526315789473


 ## B. Neural Network: MultiClass Classification 

Modify the previous architecture to model multi-class classification task. Test your architecture on the **Statlog (Vehicle Silhouettes)** Data Set ('Vehicles.csv'). Save your solution as a seperate notebook file with appropriate filename.

**Note:**
1. Perform the train/validate/test split as 70/15/15.
2. Use Random seed as '777' wherever needed.
3. Report appropriate measures in addition to accuracy and also plot the confusion matrix.

More details on the dataset can be found at: https://archive.ics.uci.edu/ml/datasets/Statlog+%28Vehicle+Silhouettes%29

In [103]:
# write your code here
from sklearn.preprocessing import OneHotEncoder
from scipy.special import softmax
from sklearn.compose import ColumnTransformer 
import pandas as pd
import requests
import io
import sklearn.metrics as ms

# train[category_cols] = train[category_cols].apply(categorize_label, axis=0)

# url = "https://raw.githubusercontent.com/Tar-ali/NN/main/vehicle.csv"
# url2 = "https://raw.githubusercontent.com/Tar-ali/NN/main/vehicle2.csv"
url3 = "https://raw.githubusercontent.com/Tar-ali/NN/main/vehicle3.csv"

#column_names = ["COMPACTNESS", "CIRCULARITY", "DISTANCE_CIRCULARITY", "RADIUS_RATIO", "PR.AXIS_ASPECT_RATIO", "MAX.LENGTH_ASPECT_RATIO", "SCATTER_RATIO", "ELONGATEDNESS", "PR.AXIS_RECTANGULARITY", "MAX.LENGTH_RECTANGULARITY", "SCALED_VARIANCE_MA", "SCALED_VARIANCE_MI", "SCALED_RADIUS_OF_GYRATION", "SKEWNESS_ABOUT_MA", "SKEWNESS_ABOUT_MI", "KURTOSIS_ABOUT_MI", "KURTOSIS_ABOUT_MA", "RATIO", "CLASS"]
data = pd.read_csv(url3)

encoder = OneHotEncoder(sparse=False)
enc = OneHotEncoder(handle_unknown='ignore')
data["car"] = data["car"].astype('category')
y = enc.fit_transform(data[['car']]).toarray()

#print(y)

#data["data"] = encoder.fit_transform(data["data"])
# ct = ColumnTransformer([("car", OneHotEncoder(),[18])], remainder="passthrough")
# data2 = ct.fit_transform(data) 
#onehotencoder = OneHotEncoder(categorical_features = [18])
#x = onehotencoder.fit_transform(data).toarray()

#y = data.car.values
# y = encoder.fit_transform(y)
#print(data["car"].unique)

del data["car"]
X = data.values.astype(np.float)

train_data, test_data, train_labels, test_labels = train_test_split(X, y, test_size=0.15, random_state=42)

scaler = preprocessing.StandardScaler().fit(train_data)
train_data = scaler.transform(train_data)
test_data = scaler.transform(test_data)

train_data, val_data, train_labels, val_labels = train_test_split(train_data, train_labels, test_size=0.17647, random_state=42)

trainx = train_data.T
trainy = train_labels.T

testx = test_data.T
testy =test_labels.T

#trainx.shape, trainy.shape, testx.shape, testy.shape

X=trainx
Y=trainy

#------------------------------------------------------------------------------


def model_architecture(X, Y):

    n_x = X.shape[0] # size of input layer
    n_h = 3
    n_y = Y.shape[0] # size of output layer
   
    return (n_x, n_h, n_y)


#------------------------------------------------------------------------------

def initialize_parameters(n_x, n_h, n_y):
    
    np.random.seed(777)

    
    ### START CODE HERE ###
    W1 = np.random.randn(n_h, n_x)*0.01
    b1 = np.zeros((n_h, 1))
    W2 = np.random.randn(n_y, n_h)*0.01
    b2 = np.zeros((n_y, 1))
    ### END CODE HERE ###
    
    assert (W1.shape == (n_h, n_x))
    assert (b1.shape == (n_h, 1))
    assert (W2.shape == (n_y, n_h))
    assert (b2.shape == (n_y, 1))
    
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    
    return parameters


#------------------------------------------------------------------------------

def forward_propagation(X, parameters):
    """
    Argument:
    X -- input data of size (n_x, m)
    parameters -- python dictionary containing your parameters (output of initialization function)
    
    Returns:
    A2 -- The sigmoid output of the second activation
    cache -- a dictionary containing "Z1", "A1", "Z2" and "A2"
    """
    # Retrieve each parameter from the dictionary "parameters"
    ### START CODE HERE ### 
    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']
    ### END CODE HERE ###
    
    # Implement Forward Propagation to calculate A2 (probabilities)
    ### START CODE HERE ### 
    Z1 = np.dot(W1, X) + b1
    A1 = np.tanh(Z1)
    Z2 = np.dot(W2, A1) + b2
    A2 = softmax(Z2, axis= 0)
    #A2 = softmax(Z2, axis=0)
    # global count 
    # count+=1
    # for i in range(A2.shape[1]):
    #   #print(A2.shape[1])
    #   print(A2[:, i])
    #   print(np.argmax(A2[:, i]))
    ### END CODE HERE ###
    # print("A2", A2.shape)
    # print("Z2", Z2.shape)
    # print(1, X.shape[1])
    
    # assert(A2.shape == (1, X.shape[1]))
    
    cache = {"Z1": Z1,
             "A1": A1,
             "Z2": Z2,
             "A2": A2}
    
    return A2, cache


#------------------------------------------------------------------------------


def backprop(parameters, cache, X, Y):
    """
    Arguments:
    parameters -- python dictionary containing our parameters 
    cache -- a dictionary containing "Z1", "A1", "Z2" and "A2".
    X -- input data
    Y -- "true" labels
    
    Returns:
    grads -- python dictionary containing your gradients with respect to different parameters
    """
    m = X.shape[1]
    
    # First, retrieve W1 and W2 from the dictionary "parameters".
    ### START CODE HERE ### 
    W1 = parameters['W1']
    W2 = parameters['W2']
    ### END CODE HERE ###
        
    # Retrieve also A1 and A2 from dictionary "cache".
    ### START CODE HERE ### 
    A1 = cache['A1']
    A2 = cache['A2']
    ### END CODE HERE ###
    
    # Backward propagation: calculate dW1, db1, dW2, db2. 
    ### START CODE HERE ### 
    dZ2 = A2-Y
    dW2 = (1/m) * np.dot(dZ2, A1.T)
    db2 = (1/m) * np.sum(dZ2, axis=1, keepdims=True)
    dZ1 = np.multiply(np.dot(W2.T, dZ2), 1 - np.power(A1, 2))
    dW1 = (1/m) * np.dot(dZ1, X.T) 
    db1 = (1/m)*np.sum(dZ1, axis=1, keepdims=True)
    ### END CODE HERE ###
    
    grads = {"dW1": dW1,
             "db1": db1,
             "dW2": dW2,
             "db2": db2}
    
    return grads


#------------------------------------------------------------------------------


def update(parameters, grads, learning_rate = 0.01):
    """
    Arguments:
    parameters -- python dictionary containing your parameters 
    grads -- python dictionary containing your gradients 
    learning_rate -- The learning rate
    
    Returns:
    parameters -- python dictionary containing your updated parameters 
    """
    # Retrieve each parameter from the dictionary "parameters"
    ### START CODE HERE ### 
    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']
    ### END CODE HERE ###
    
    # Retrieve each gradient from the dictionary "grads"
    ### START CODE HERE ### 
    dW1 = grads['dW1']
    db1 = grads['db1']
    dW2 = grads['dW2']
    db2 = grads['db2']
    ## END CODE HERE ###
    
    # Update rule for each parameter
    ### START CODE HERE ### 
    W1 = W1 - learning_rate * dW1
    b1 = b1 - learning_rate * db1
    W2 = W2 - learning_rate * dW2
    b2 = b2 - learning_rate * db2
    ### END CODE HERE ###
    
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    
    return parameters


#------------------------------------------------------------------------------


def compute_loss(A2,Y):
    """
    X is the output from fully connected layer (num_examples x num_classes)
    y is labels (num_examples x 1)
    	Note that y is not one-hot encoded vector. 
    	It can be computed as y.argmax(axis=1) from one-hot encoded vectors of labels if required.
    """
    m = Y.shape[0]
    p =  A2     #softmax(A2)
    log_likelihood = -np.log(p[range(m),Y.argmax(axis=1)])
    loss = np.sum(log_likelihood) / m
    return loss


#------------------------------------------------------------------------------


def NeuralNetwork(X, Y, n_h, num_iterations = 10000, learning_rate = 0.01, print_loss=False):
    """
    Arguments:
    X -- dataset
    Y -- labels 
    n_h -- size of the hidden layer
    num_iterations -- Number of iterations in gradient descent loop
    learning_rate -- The learning rate
    print_loss -- if True, print the loss every 1000 iterations
    
    Returns:
    parameters -- parameters learnt by the model. They can then be used to make predictions.
    """
    
    np.random.seed(777)
    n_x = model_architecture(X, Y)[0]
    n_y = model_architecture(X, Y)[2]
    
    # Initialize parameters
    ### START CODE HERE ### 
    parameters = initialize_parameters(n_x, n_h, n_y)
    ### END CODE HERE ###
    
    # Loop (gradient descent)

    for i in range(0, num_iterations):
         
        ### START CODE HERE ### 
        # Forward propagation. Inputs: "X, parameters". Outputs: "A2, cache".
        A2, cache = forward_propagation(X, parameters)
        
        # loss function. Inputs: "A2, Y, parameters". Outputs: "loss".
        loss = compute_loss(A2, Y)
 
        # Backpropagation. Inputs: "parameters, cache, X, Y". Outputs: "grads".
        grads = backprop(parameters, cache, X, Y)
 
        # Gradient descent parameter update. Inputs: "parameters, grads". Outputs: "parameters".
        parameters =  update(parameters, grads)
        
        ### END CODE HERE ###
        
        # Print the loss every 100 iterations
        if print_loss and i % 100 == 0:
            print ("loss after iteration %i: %f" %(i, loss))

    return parameters


#------------------------------------------------------------------------------

def predict(parameters, X):
    """
    Arguments:
    parameters -- python dictionary containing your parameters 
    X -- input data 
    
    Returns
    predictions -- vector of predictions of our model
    """
    


    # Computes probabilities using forward propagation, and classifies to 0/1 using 0.5 as the threshold.
    ### START CODE HERE ### 
    A2, cache = forward_propagation(X, parameters)
    predictions =  A2 > 0.5 #np.argmax(A2, axis=1)
    #print(A2[:, 1])

    # for i in range(A.shape[1]):
        
    #     # Convert probabilities A[0,i] to actual predictions p[0,i]
  
    #     if(A[0,i]< 0.5):
    #         Y_prediction[0,i] = 0
            
    #     else:
    #         Y_prediction[0,i] = 1


    ### END CODE HERE ###
    
    return predictions

parameters = NeuralNetwork(X, Y, n_h = 4, num_iterations = 10000, print_loss=True)
forward_propagation(X, parameters)

print()
print("accuracy:")
predictions = predict(parameters, X)
print(accuracy_score(Y.T, predictions.T))



#print(train_data.shape)


loss after iteration 0: 1.386366
loss after iteration 100: 1.386218
loss after iteration 200: 1.385995
loss after iteration 300: 1.385304
loss after iteration 400: 1.383247
loss after iteration 500: 1.377576
loss after iteration 600: 1.363864
loss after iteration 700: 1.337624
loss after iteration 800: 1.299827
loss after iteration 900: 1.256504
loss after iteration 1000: 1.212583
loss after iteration 1100: 1.169717
loss after iteration 1200: 1.127703
loss after iteration 1300: 1.086121
loss after iteration 1400: 1.044965
loss after iteration 1500: 1.004199
loss after iteration 1600: 0.963538
loss after iteration 1700: 0.922401
loss after iteration 1800: 0.879719
loss after iteration 1900: 0.834718
loss after iteration 2000: 0.788729
loss after iteration 2100: 0.744388
loss after iteration 2200: 0.703631
loss after iteration 2300: 0.667270
loss after iteration 2400: 0.635353
loss after iteration 2500: 0.607523
loss after iteration 2600: 0.583272
loss after iteration 2700: 0.562077
loss

In [127]:
predictions_test = predict(parameters, testx) 
print(accuracy_score(testy.T, predictions_test.T))

labelP = np.argmax(predictions_test.T, axis = 1)
labelR = np.argmax(testy.T, axis = 1)


print(ms.confusion_matrix(labelP,labelR))

0.8031496062992126
[[38  0  7  1]
 [ 0 15  8  0]
 [ 0  5 21  0]
 [ 0  1  1 30]]


References:
- https://www.coursera.org/learn/neural-networks-deep-learning
- http://scs.ryerson.ca/~aharley/neural-networks/
- http://cs231n.github.io/neural-networks-case-study/
- https://archive.ics.uci.edu/ml/datasets.php